In [1]:
import numpy as np
import os
import tensorflow as tf
import datetime

In [2]:
class MLP:
    def __init__(self,data_file, log_file, save_file, hidden_node_num):
        #Define instance variable
        self.__data_file = data_file
        self.__log_file = log_file
        self.__save_file = save_file
        self.__log = None

        self.__hidden_node_num = hidden_node_num
        
        self.x_train_data = None
        self.y_train_data = None
        self.x_test_data = None
        self.y_test_data = None
        self.__x_data_len = None
        
        self.__y = None
        self.__x = None
        
        self.__w1 = None
        self.__w2 = None
        self.__b1 = None
        self.__b2 = None
        self.__cost = None
        
        self.__y_out = None
        self.__train = None
        self.__sess = None
        self.__saver = None
        
        self.__loadData()
        self.__init_model()
        
    def __loadData(self):
        #Read data file
        data_file_name = self.__data_file
        xy = np.genfromtxt(data_file_name, dtype='float32')
        
        #Shuffle data
        np.random.shuffle(xy)
        
        #Data Split into train data and test data
        all_data_num = xy[:,1:-1].shape[0]
        train_data_num = int(all_data_num * 95 /100)
        test_data_num = all_data_num - train_data_num
        self.x_train_data =  xy[:train_data_num,1:-1]
        self.y_train_data =  xy[:train_data_num,-1]
        self.x_test_data =  xy[train_data_num:,1:-1]
        self.y_test_data =  xy[train_data_num:,-1]

        #Transpose Matrix having x_data
        self.x_train_data = self.x_train_data.transpose()
        self.x_test_data = self.x_test_data.transpose()

        #Calculate num of variable
        self.__x_data_len = len(self.x_train_data)
        
    def __init_model(self):
        self.__x = tf.placeholder(dtype=tf.float32)
        self.__y = tf.placeholder(dtype=tf.float32)
        
        # setup hidden layer1
        self.__w1 = tf.Variable(tf.random_normal([self.__hidden_node_num, self.__x_data_len]), name='w1')
        self.__b1 = tf.Variable(tf.random_normal([self.__hidden_node_num, 1]), name='b1')
        hidden_layer1 = tf.nn.sigmoid(tf.matmul(self.__w1, self.__x ) + self.__b1)
        
        # setup output layer
        self.__w2 = tf.Variable(tf.random_normal([1,self.__hidden_node_num]),name='w2')
        self.__b2 = tf.Variable(tf.random_normal([1,1]), name='b2')
        self.__y_out = tf.matmul(self.__w2,hidden_layer1) + self.__b2
        
        # setup cost function and optimizer
        self.__cost = tf.reduce_mean(tf.square(self.__y_out-  self.__y))
        opt = tf.train.GradientDescentOptimizer(learning_rate=0.01)
        # self.__cost = tf.nn.l2_loss(self.__y_out-self.__y)
        # opt= tf.train.AdamOptimizer(0.1)
        self.__train = opt.minimize(self.__cost)
    
    def init_sess(self):
        init = tf.global_variables_initializer()
        self.__sess = tf.Session()
        self.__sess.run(init)
        
        if self.__save_file != None:
            self.__saver = tf.train.Saver()
            
        if self.__log_file != None:
            self.__log = open(self.__log_file,'w')
            self.__log.write("Step\tTraning Cost\tTest Accuracy\n")
            self.__log.close()
        
    def model_train(self):
        self.__sess.run(self.__train,feed_dict={self.__x: self.x_test_data, self.__y: self.y_test_data})
        
    def model_save(self):
        if self.__save_file != None:
            self.__saver.save(self.__sess, self.__save_file)

    def log_write(self, num):
        if self.__log_file != None:
            predic = self.__sess.run(self.__y_out,feed_dict={self.__x: self.x_test_data})
            result = (self.y_test_data- predic) /predic * 100
            self.__log = open(self.__log_file,'a')
            self.__log.write(str(num) + "\t")
            self.__log.write(str(self.__sess.run(self.__cost,feed_dict={self.__x: self.x_test_data,self.__y: self.y_test_data}))+ "\t")
            self.__log.write(str(np.mean(result))+ "\n")
            self.__log.close()
        
    def close(self):
        self.__sess.close();

In [3]:
hidden_node_num = 10;
train_num = 25000
print_inter = 5000 
alpha_num = 1
start_alpha = 0
end_alpha = 0

In [4]:
nowDatetime = datetime.datetime.now().strftime('%m%d_%H%M')
logDir = "log/" + nowDatetime
saveDir= "model/" + nowDatetime
if not os.path.exists(logDir):
    os.makedirs(logDir)
    
if not os.path.exists(saveDir):
    os.makedirs(saveDir)    

model_array = []

for i in range(start_alpha,end_alpha+1):
    dataPath = "data/data_Alpha_" + str(i) + ".txt"
    logPath = logDir + "/log_" + str(i) +".txt"
    savePath = saveDir  + "/model_" + str(i)
    model_array.append(MLP(dataPath, logPath, savePath,hidden_node_num))

In [ ]:
print("Traning Start")

for i in range(alpha_num):
    model_array[i].init_sess()
    
for step in range(train_num):
    for i in range(alpha_num):
        model_array[i].model_train()
    if step%print_inter == 0:
        print("Step : ",step)
        for i in range(alpha_num):
            model_array[i].log_write(step)
            model_array[i].model_save()
            
for i in range(alpha_num):            
    model_array[i].log_write(step)
    model_array[i].model_save()            
    model_array[i].close()
    
print("Traning Finish")

Traning Start
Step :  0
